In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


def crawling(soup) :
    movie = []
    
    # 영화 이름 , 포스터
    
    div = soup.find("div", class_="aside NE=a:mvi")
    if div == None :
        return
    else :
        poster = div.find("span", class_="pic_area")
        movie_poster = poster.find("img")["src"]
        movie_name = poster.find("img")["alt"]
        
        movie.append(movie_name)
        movie.append(movie_poster)
        
    # 영화 평점

    star = soup.find("div",class_="score_area")
    if star == None :
        return
    else :
        score = star.find("em").get_text()
        
        movie.append(score)
    
    
#     #장르,나라,상영시간,개봉날짜, 영화 감독, 출연, 등급
    
    ul = soup.find("ul", class_= "end_info NE=a:mvi")
    
    if ul == None :
        return
    else :
        
        li = ul.find("li", class_="info_lst").find_all("li")

        for info in li :
            movie.append(info.text.replace("\n","").replace("기본정보","").replace("\t","").replace("개봉일","").replace("장르","")
                     .replace("감독","").replace("주연",""))


            
        del movie[-1]

    
#     # 줄거리
    
    story = soup.find("div", class_="end_dsc _close NE=a:mvi")
    movie_story = story.get_text().replace("\n","").replace("\t","").replace("\xa0"," ").replace("줄거리더보기"," ")
    
    movie.append(movie_story)
    
    
#     print(movie_story)
#     print(movie_name)
#     print(score)
#    print(mo)
    
    return movie


def get_href(soup) :
    # 각 영화별 href를 리스트로 반환
    result =[]
    div = soup.find("div",class_="lst_thum_wrap")
    a = div.find_all("a", class_="NPI=a:content")
    
    for href in a :
        result.append("https://serieson.naver.com/" + href["href"])

    return result
 

def main() :
    list_href = []
    result = []
    movie_columns = ['제목','포스터','평점','나라,상영시간','개봉일','등급','장르','감독','주연','줄거리']
    url = "https://serieson.naver.com/movie/categoryList.nhn?categoryCode=ALL&orderType=sale&sortingType=&mobileYn=&drmFreeYn=&freeYn=&discountYn=&tagCode=&page="
    for i in range(1,1000):
        req = requests.get(url+str(i))

        soup = BeautifulSoup(req.text, "html.parser")
        
    
        list_href.append( get_href(soup) )
    
    list_href= sum(list_href, [])
    

    for href in list_href :
        href_req = requests.get(href)
        href_soup = BeautifulSoup(href_req.text, "html.parser")
        append_result = crawling(href_soup)
        if append_result!=None :
            result.append(append_result)
        

    movie_df = pd.DataFrame(result)
    movie_df.to_csv("navermovies2222.csv")
    print(movie_df)


if __name__ == "__main__" :
    main()

    

                                        0  \
0                                 너의 이름은.   
1                                    라라랜드   
2                                      증인   
3                                    청년경찰   
4                                       돈   
...                                   ...   
17315       알프레드 브렌델 리스트, '순례의 해' 제1권 스위스   
17316                         브람스, 독일 진혼곡   
17317  레너드 번스타인 비엔나 필하모닉 오케스트라 슈만, 교향곡 4번   
17318                     롤란도 빌라존의 헨델 아리아   
17319                                파이팅맨   

                                                       1     2           3  \
0      https://movie-phinf.pstatic.net/20171222_70/15...  8.78    일본, 106분   
1      https://movie-phinf.pstatic.net/20161123_188/1...   8.6    미국, 127분   
2      https://movie-phinf.pstatic.net/20190226_194/1...  9.09  대한민국, 129분   
3      https://movie-phinf.pstatic.net/20170814_131/1...  8.26  대한민국, 109분   
4      https://movie-phinf.pstatic.net/20190306_280/1...  7.41  대한민국,

In [ ]:
movie_columns = ['순위','제목','포스터','평점','나라,상영시간','개봉일','등급','장르','감독','주연','줄거리']
df = pd.read_csv("navermovies2222.csv" , sep=",")
df.columns = movie_columns
df.to_csv("navermovies2.csv")

,Unnamed: 0,순위,제목,포스터,평점,"나라,상영시간",개봉일,등급,장르,감독,주연,줄거리
0,0,0,너의 이름은.,https://movie-phinf.pstatic.net/20171222_70/15...,8.78,"일본, 106분",2017.01.04.,12세 관람가,드라마,신카이 마코토,"지창욱, 김소현, 이레, 카미키 류노스케, 카미시라이시 모네, 타니 카논","꿈 속에서 몸이 뒤바뀐 도시 소년 ‘타키’와 시골 소녀 ‘미츠하’, 만난 적 없는 ..."
1,1,1,라라랜드,https://movie-phinf.pstatic.net/20161123_188/1...,8.60,"미국, 127분",2016.12.07.,12세 관람가,드라마,다미엔 차젤레,"엠마 스톤, 라이언 고슬링",꿈을 꾸는 사람들을 위한 별들의 도시 ‘라라랜드’. 재즈 피아니스트 ‘세바스찬’(라...
2,2,2,증인,https://movie-phinf.pstatic.net/20190226_194/1...,9.09,"대한민국, 129분",2019.02.13.,12세 관람가,드라마,이한,"정우성, 김향기",“목격자가 있어. 자폐아야” 신념은 잠시 접어두고 현실을 위해 속물이 되기로 마음...
3,3,3,청년경찰,https://movie-phinf.pstatic.net/20170814_131/1...,8.26,"대한민국, 109분",2017.08.09.,15세 관람가,액션,김주환,"박서준, 강하늘",의욕충만 경찰대생 기준(박서준) X 이론백단 경찰대생 희열(강하늘) 둘도 없는 친구...
4,4,4,돈,https://movie-phinf.pstatic.net/20190306_280/1...,7.41,"대한민국, 115분",2019.03.20.,15세 관람가,액션,박누리,"류준열, 유지태, 조우진",오직 부자가 되고 싶은 꿈을 품고 여의도 증권가에 입성한 신입 주식 브로커 조일현(...
...,...,...,...,...,...,...,...,...,...,...,...,...
17315,17315,17315,"알프레드 브렌델 리스트, '순례의 해' 제1권 스위스",https://movie-phinf.pstatic.net/20150618_258/1...,0.00,"독일, 50분",NaN,전체 관람가,공연실황,유니텔 클래시카,알프레드 브렌델,오스트리아의 피아니스트 알프레드 브렌델은 1931년에 태어나 20세기 중반부터 국제...
17316,17316,17316,"브람스, 독일 진혼곡",https://movie-phinf.pstatic.net/20150616_79/14...,0.00,"독일, 83분",NaN,전체 관람가,공연실황,유니텔 클래시카,"크리스티안 틸레만, 뮌헨 필하모닉 오케스트라, 바바리안 라디오 합창단, 크리스틴 셰퍼","요하네스 브람스 (1833-1897) 의 `독일 진혹곡’이, 2007년 4월에 독일..."
17317,17317,17317,"레너드 번스타인 비엔나 필하모닉 오케스트라 슈만, 교향곡 4번",https://movie-phinf.pstatic.net/20150615_51/14...,0.00,"독일, 36분",NaN,전체 관람가,공연실황,유니텔 클래시카,"레너드 번스타인, 비엔나 필하모닉 오케스트라","유니텔은 말러와 베토벤, 브람스의 관현악 작품에 이어서, 1985년 무렵 로베르트 ..."
17318,17318,17318,롤란도 빌라존의 헨델 아리아,https://movie-phinf.pstatic.net/20150616_90/14...,0.00,"독일, 76분",NaN,전체 관람가,공연실황,유니텔 클래시카,"폴 맥크리쉬, 브리엘리 플레이어즈","롤란도 빌라존만큼 유혹적이고 열정적이며, 베르디나 푸치니와 같은 위대한 낭만파 음악..."
